In [64]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp


In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss


### Read data

In [66]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../data/MMasseyOrdinals.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')

kp_rankings = pd.read_csv('../data/kenpom_pre_tourney_snapshot.csv')

regular_season_results_w = pd.read_csv('../data/WRegularSeasonDetailedResults.csv')
detailed_tourney_results_w = pd.read_csv('../data/WNCAATourneyDetailedResults.csv')

sub_df = pd.read_csv('../data/SampleSubmission2023.csv')
mteams = pd.read_csv('../data/MTeams.csv')
wteams = pd.read_csv('../data/WTeams.csv')

seeds_w = pd.read_csv('../data/WNCAATourneySeeds.csv')

M538 = pd.read_csv('../data/M538.csv')
W538 = pd.read_csv('../data/W538.csv')

seed_round = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

first_round_odds_data = pd.read_csv('../data/first_round_odds_data.csv')



### Set Up Data

In [67]:
to_predict_mens, to_predict_womens, regular_season_games, regular_season_games_w = preprocess.full_setup(detailed_tourney_results, regular_season_results,
               detailed_tourney_results_w, regular_season_results_w,
               sub_df, mteams)

### Add Features

In [68]:
to_predict_womens = feature_engineering.TournamentSeed(tourney_seeds=seeds_w).add(to_predict_womens)
to_predict_womens = feature_engineering.Efficiency(games=regular_season_games_w, away_bonus=0).add(to_predict_womens)
to_predict_womens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=W538).add(to_predict_womens)

In [69]:
to_predict_mens = feature_engineering.TeamNames(mteams).add(to_predict_mens)
to_predict_mens = feature_engineering.FirstRoundOdds(first_round_odds_data).add(to_predict_mens)

In [70]:
to_predict_mens = feature_engineering.SeasonStats(regular_season_games).add(to_predict_mens)
to_predict_mens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=M538).add(to_predict_mens)
to_predict_mens = feature_engineering.PreSeasonAPRankings(rankings_df=rankings).add(to_predict_mens)
to_predict_mens = feature_engineering.TournamentSeed(tourney_seeds=seeds).add(to_predict_mens)
to_predict_mens = feature_engineering.Efficiency(games=regular_season_games, away_bonus=3).add(to_predict_mens)
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='SAG').add(to_predict_mens)
to_predict_mens = feature_engineering.Kenpom(kp_snapshot=kp_rankings).add(to_predict_mens)

# this one takes 3 minutes to run
# to_predict_mens = feature_engineering.TeamQuality(games=regular_season_games).add(to_predict_mens)

to_predict_mens = feature_engineering.RoundNumber(seeds, seed_round).add(to_predict_mens)



In [71]:
to_predict_mens.head()

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,...,t2_luck,t2_luck_rank,t2_sos_adjem,t2_sos_adjem_rank,t2_sos_opp_o,t2_sos_opp_o_rank,t2_sos_opp_d,t2_sos_opp_d_rank,t2_ncsos_adjem_rank,GameRound
0,Historical,2003_1421_1411,None,2003,1421,1411,1,M,8,UNC Asheville,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Historical,2003_1112_1436,None,2003,1112,1436,1,M,29,Arizona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,Historical,2003_1113_1272,None,2003,1113,1272,1,M,13,Arizona St,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,Historical,2003_1141_1166,None,2003,1141,1166,1,M,6,C Michigan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,Historical,2003_1143_1301,None,2003,1143,1301,1,M,2,California,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### Split Dataset

In [72]:
first_round_df = to_predict_mens[to_predict_mens.GameRound == 1].copy()
other_rounds_df = to_predict_mens[to_predict_mens.GameRound > 1].copy()

In [73]:
other_rounds_df = feature_engineering.FirstRoundOpponentQuality(first_round_df, other_rounds_df).add()


In [74]:
other_rounds_df.head()

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,...,t2_sos_adjem,t2_sos_adjem_rank,t2_sos_opp_o,t2_sos_opp_o_rank,t2_sos_opp_d,t2_sos_opp_d_rank,t2_ncsos_adjem_rank,GameRound,round1_opponent_rank,round1_opponent_quality
0,Historical,2003_1112_1211,None,2003,1112,1211,1,M,1,Arizona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,72.542565,0.290757
1,Historical,2003_1163_1390,None,2003,1163,1390,1,M,11,Connecticut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,85.258089,0.683338
2,Historical,2003_1181_1141,None,2003,1181,1141,1,M,26,Duke,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,77.955307,0.457871
3,Historical,2003_1242_1113,None,2003,1242,1113,1,M,32,Kansas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,76.909200,0.425573
4,Historical,2003_1266_1281,None,2003,1266,1281,1,M,9,Marquette,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,79.972665,0.520155


In [75]:
first_round_df.to_csv("../development_notebooks/to_predict_mens_first_round_away_bonus_3.csv")
other_rounds_df.to_csv("../development_notebooks/to_predict_mens_other_rounds_away_bonus_3.csv")